<a href="https://colab.research.google.com/github/rakib3004/SPL3/blob/main/UMLJsonConverterFromCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Software System Reconstruction with LLMs
Author: Md. Rakib Trofder  
Supervisor:Denys Poshyvanyk                                      
Mentor:Daniel Rodriguez-Cardenas


In [2]:
import re
import json

def parse_java_code(java_code):
    class_relationships = []

    class_pattern = r"(public\s+)?class\s+(\w+)\s*{([^}]+)}"

    implements_pattern = r"(public\s+)?class\s+(\w+)\s+implements\s+(\w+)"
    extends_pattern = r"(public\s+)?class\s+(\w+)\s+extends\s+(\w+)"

    implements_matches = re.findall(implements_pattern, java_code)
    for _, implementing_class, implemented_interface in implements_matches:
        class_relationships.append({
            "class": implementing_class,
            "relationship_type": "Realization",
            "with": implemented_interface
        })

    extends_matches = re.findall(extends_pattern, java_code)
    for _, extending_class, extended_class in extends_matches:
        class_relationships.append({
            "class": extending_class,
            "relationship_type": "Generalization",
            "with": extended_class
        })


    classes = {}
    associations = []

    class_pattern = r"(public\s+)?class\s+(\w+)\s*.*?{([^}]+)}"
    class_title_pattern =  r"(public\s+)?class\s+(\w+)"
    field_pattern = r"(\w+)\s+(\w+)\s*=\s*new\s+(\w+)\(\);"

    class_matches = re.findall(class_title_pattern, java_code)
    class_names = [match[1] for match in class_matches]


    class_matches = re.findall(class_pattern, java_code, re.DOTALL)
    for class_match in class_matches:
        _, class_name, class_body = class_match

        field_matches = re.findall(field_pattern, class_body)
        for field_match in field_matches:
            field_type, field_name, _ = field_match
            class_relationships.append({
                "class": class_name,
                "relationship_type": "Association",
                    "with": field_type,
                          }
            )

    classes_info = []
    methods_info = []
    imports_info = []
    packages_info = []
    class_info = None
    lines = java_code.split("\n")
    is_in_method = False
    package_name=""

    for line in lines:
        if line.strip().startswith("package "):
            package_name = line.split(" ")[1][:-1]
            packages_info.append(package_name)
        elif line.strip().startswith("import "):
            import_name = line.split(" ")[1][:-1]
            imports_info.append(import_name)
        elif line.strip().startswith("public class "):

            class_name = line.split(" ")[2].split("{")[0]
            #print('class name', class_name)
            class_info = {
                "name": class_name,
                "methods": []
            }

        elif line.strip().startswith("public") and line.strip().endswith(") {"):
            method_signature = line.split("(")[0].split(" ")[2:]
            method_name = method_signature[-1]
            method_args = method_signature[:-1]
            #print("methods - ", method_signature)
            is_in_method = True
            method_info = {
                "name": method_name,
            }

        elif is_in_method and line.strip() == "}":
            is_in_method = False
            if method_info is not None:
              class_info["methods"].append(method_info)
              methods_info.append(method_info)


    result = {
        "package": packages_info,
        "imports": imports_info,
        "class": class_info,
        "relationships": class_relationships,

    }

    return result

if __name__ == "__main__":
    java_code = """
package Collection;

import FilePackage.DateTimeWriter;
import ObjectOriented.PriorityData;
import MultiVariableRegression.MedianCalculation;

import java.util.*;

public class PriceCollection {
    MedianCalculation medianCalculation = new MedianCalculation();

    public void priceCollectionMLRMethods(PriorityData[] priorityData, int numberOfBooks) {

        String className = this.getClass().getSimpleName();
        DateTimeWriter dateTimeWriter = new DateTimeWriter();
        dateTimeWriter.dateTimeWriterMethods(className);

        Set<String> priceID = new HashSet<>();
        int iterator1;
        for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {

            priceID.add(priorityData[iterator1].bookData.bookPrice);
        }

        System.out.println(priceID.size());
        System.out.println();
        System.out.println();
        System.out.println();
        int number = 0;
        int writerCount = 0;
        Iterator<String> iterator = priceID.iterator();

        while (iterator.hasNext()) {
            String element = iterator.next();

            int count = 0;

            number = 1;
            writerCount++;

            List<Double> list = new ArrayList<>();

            System.out.println("##########Books of \"" + element + "\" #########(" + writerCount + ")###");
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {
                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    System.out.println(number + " . " + priorityData[iterator1].bookData.bookName);
                    list.add(priorityData[iterator1].MLRweight);
                    number++;
                    count++;
                }
            }
            double summation = 0.0;

            int sizeB = list.size();
            if (sizeB > 7) {
                medianCalculation.medianCalculationMethods(list);

            }
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {

                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    summation = summation + priorityData[iterator1].MLRweight;
                }
            }
            double mean = summation / number;

            double standard_deviation = 0.0;

            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {

                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {

                    standard_deviation = standard_deviation + ((mean - priorityData[iterator1].MLRweight) *
                            (mean - priorityData[iterator1].MLRweight));
                }
            }

            standard_deviation = standard_deviation / (number - 1);
            standard_deviation = Math.sqrt(standard_deviation);
            System.out.println("Mean : " + mean + "    " + "Standard Deviation : " + standard_deviation);

        }
    }

    public void priceCollectionAHPMethods(PriorityData[] priorityData, int numberOfBooks) {

        String className = this.getClass().getSimpleName();
        DateTimeWriter dateTimeWriter = new DateTimeWriter();
        dateTimeWriter.dateTimeWriterMethods(className);

        Set<String> priceID = new HashSet<>();
        int iterator1;
        for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {
            priceID.add(priorityData[iterator1].bookData.bookPrice);
        }

        System.out.println(priceID.size());
        System.out.println();
        System.out.println();
        System.out.println();
        int number = 0;
        int writerCount = 0;
        Iterator<String> iterator = priceID.iterator();
        while (iterator.hasNext()) {
            String element = iterator.next();
            int count = 0;

            number = 1;
            writerCount++;

            List<Double> list = new ArrayList<>();
            System.out.println("##########Books of \"" + element + "\" #########(" + writerCount + ")###");
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {
                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    System.out.println(number + " . " + priorityData[iterator1].bookData.bookName);
                    list.add(priorityData[iterator1].AHPweight);
                    number++;
                    count++;
                }
            }

            double summation = 0.0;
            int sizeB = list.size();
            if (sizeB > 7) {
                medianCalculation.medianCalculationMethods(list);
            }
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {

                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    summation = summation + priorityData[iterator1].AHPweight;
                }
            }
            double mean = summation / number;
            double standard_deviation = 0.0;
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {
                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    standard_deviation = standard_deviation + ((mean - priorityData[iterator1].AHPweight) *
                            (mean - priorityData[iterator1].AHPweight));
                }
            }
            standard_deviation = standard_deviation / (number - 1);
            standard_deviation = Math.sqrt(standard_deviation);
            System.out.println("Mean : " + mean + "    " + "Standard Deviation : " + standard_deviation);
        }
    }

    public void priceCollectionPRAMethods(PriorityData[] priorityData, int numberOfBooks) {

        String className = this.getClass().getSimpleName();
        DateTimeWriter dateTimeWriter = new DateTimeWriter();
        dateTimeWriter.dateTimeWriterMethods(className);

        Set<String> priceID = new HashSet<>();
        int iterator1;

        for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {
            priceID.add(priorityData[iterator1].bookData.bookPrice);
        }
        System.out.println(priceID.size());
        System.out.println();
        System.out.println();
        System.out.println();
        int number = 0;
        int writerCount = 0;
        Iterator<String> iterator = priceID.iterator();

        while (iterator.hasNext()) {
            String element = iterator.next();

            int count = 0;
            number = 1;
            writerCount++;
            List<Double> list = new ArrayList<>();
            System.out.println("##########Books of \"" + element + "\" #########(" + writerCount + ")###");
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {
                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    System.out.println(number + " . " + priorityData[iterator1].bookData.bookName);
                    list.add(priorityData[iterator1].PRAweight);
                    number++;
                    count++;
                }
            }
            double summation = 0.0;
            int sizeB = list.size();
            if (sizeB > 7) {
                medianCalculation.medianCalculationMethods(list);
            }
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {

                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    summation = summation + priorityData[iterator1].PRAweight;
                }
            }
            double mean = summation / number;
            double standard_deviation = 0.0;
            for (iterator1 = 0; iterator1 < numberOfBooks; iterator1++) {

                if (priorityData[iterator1].bookData.bookPrice.equals(element)) {
                    standard_deviation = standard_deviation + ((mean - priorityData[iterator1].PRAweight) *
                            (mean - priorityData[iterator1].PRAweight));
                }
            }
            standard_deviation = standard_deviation / (number - 1);
            standard_deviation = Math.sqrt(standard_deviation);
            System.out.println("Mean : " + mean + "    " + "Standard Deviation : " + standard_deviation);
        }
    }
}
    """

    result = parse_java_code(java_code)
    json_output = json.dumps(result, indent=2)
    print(json_output)


{
  "package": [
    "Collection"
  ],
  "imports": [
    "FilePackage.DateTimeWriter",
    "ObjectOriented.PriorityData",
    "MultiVariableRegression.MedianCalculation",
    "java.util.*"
  ],
  "class": [],
  "class2": {
    "name": "PriceCollection",
    "methods": [
      {
        "name": "priceCollectionMLRMethods"
      },
      {
        "name": "priceCollectionAHPMethods"
      },
      {
        "name": "priceCollectionPRAMethods"
      }
    ]
  },
  "relationships": [
    {
      "class": "PriceCollection",
      "relationship_type": "Association",
      "with": "MedianCalculation"
    },
    {
      "class": "PriceCollection",
      "relationship_type": "Association",
      "with": "DateTimeWriter"
    }
  ]
}
